In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

newdata = '/content/drive/MyDrive/deeplearningProgression/Newdata.csv'
newdata = pd.read_csv(newdata)

print(newdata.head())

'''
backtranslate = '/content/drive/MyDrive/deeplearningProgression/backtranslate.csv'
backtranslate = pd.read_csv(backtranslate)

huggingFace_dataset = '/content/drive/MyDrive/deeplearningProgression/huggingface_dataset.csv'
huggingFace_dataset = pd.read_csv(huggingFace_dataset)


print(backtranslate.head())
print(huggingFace_dataset.head())
'''

Mounted at /content/drive
                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  label  
0  December 31, 2017      1  
1  December 31, 2017      1  
2  December 30, 2017      1  
3  December 29, 2017      1  
4  December 25, 2017      1  


"\nbacktranslate = '/content/drive/MyDrive/deeplearningProgression/backtranslate.csv'\nbacktranslate = pd.read_csv(backtranslate)\n\nhuggingFace_dataset = '/content/drive/MyDrive/deeplearningProgression/huggingface_dataset.csv'\nhuggingFace_dataset = pd.read_csv(huggingFace_dataset)\n\n\nprint(backtranslate.head())\nprint(huggingFace_dataset.head())\n"

In [4]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

# Download necessary NLTK models and corpora
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# 1. Remove any unnamed columns (if present)
newdata = newdata.loc[:, ~newdata.columns.str.contains('^Unnamed')]

# 2. Handle missing values by creating new columns and replacing missing values with "missing"
newdata['news_missing'] = newdata['text'].isna() | newdata['text'].eq('')
newdata['label_missing'] = newdata['label'].isna() | newdata['label'].eq('')

newdata ['text'] = newdata['text'].replace('', 'missing')
newdata ['text'] = newdata ['text'].fillna('missing')

# 3. Convert all text to lowercase
newdata ['text'] = newdata['text'].str.lower()

# 4. Remove punctuation
def remove_punctuation(text):
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    return text

newdata ['text'] = newdata ['text'].apply(remove_punctuation)

# 5. Remove stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    if isinstance(text, str):
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
        return ' '.join(filtered_text)
    return text

newdata['text'] = newdata['text'].apply(remove_stopwords)

# 6. Tokenization
def tokenize_text(text):
    return [word for word in word_tokenize(text) if word.isalnum()]

newdata['text_tokens'] = newdata['text'].apply(tokenize_text)

# 7. Lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, str):
        word_tokens = word_tokenize(text)
        lemmatized_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokens if word.lower() not in stop_words]
        return ' '.join(lemmatized_tokens)
    return text

newdata['text'] = newdata['text'].apply(lemmatize_text)

# Save the processed dataset to a new CSV file
newdata.to_csv('new_dataset_processed.csv', index=False)

# Check the quality of the cleaned dataset
print(newdata.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  donald trump wish american happy new year leav...    News   
1  house intelligence committee chairman devin nu...    News   
2  friday revealed former milwaukee sheriff david...    News   
3  christmas day donald trump announced would bac...    News   
4  pope francis used annual christmas day message...    News   

                date  label  news_missing  label_missing  \
0  December 31, 2017      1         False          False   
1  December 31, 2017      1         False          False   
2  December 30, 2017      1         False          False   
3  December 29, 2017      1         Fa

In [5]:
# Count the occurrences of each label
label_counts = newdata['label'].value_counts()

# Print the counts
print(f"Number of fake news (label=1): {label_counts[1]}")
print(f"Number of true news (label=0): {label_counts[0]}")


Number of fake news (label=1): 23481
Number of true news (label=0): 21417
